<h1 style="text-align:center;">Uploading Firmware to the iLO Repository For Flashing</h1>

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This Jupyter Notebook will contains a example code to demonstrate of Uploading Firmware to the iLO Repository For Flashing. This code is written in Python and uses the redfish library to perform the required operations.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this u need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
    </ul>
</div>

In [ ]:
import os
import sys
import json
import argparse
from random import randint

from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
import logging
from redfish import redfish_logger

from ilorest_util import get_resource_directory

In [ ]:
LOGGERFILE = "RedfishApiExamples.log"
LOGGERFORMAT = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
LOGGER = redfish_logger(LOGGERFILE, LOGGERFORMAT, logging.DEBUG)

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 2. This is about upload_firmware function</h3>
        
1. The function upload_firmware takes in parameters such as _redfishobj, firmware_loc, comp_type, update_repo, and update_target..<br>
<br>
    
2. It uploads firmware to a specified location using the Redfish API and HTTP POST request.<br>
    <br>

3. The function first obtains the UpdateService URI and HttpPushUri using the _redfishobj and then reads the firmware file as binary data.<br>
    <br>

4. It creates a payload containing the session key, JSON data, and the firmware file in multipart-form data format..<br>
    <br>
5.   Finally, the function sends the payload as an HTTP POST request to the specified path and returns an appropriate response based on the HTTP status code.
<br>
    



 </div>

In [1]:
def upload_firmware(_redfishobj, firmware_loc, comp_type, update_repo=True, update_target=False):
    resource_instances = get_resource_directory(_redfishobj)

    if DISABLE_RESOURCE_DIR or not resource_instances:
        #resource directory is not available so we will navigate through paths manually
        update_service_uri = _redfishobj.root.obj['UpdateService']['@odata.id']
    else:
        #obtain all account instances from resource directory
        for instance in resource_instances:
            if '#UpdateService.' in instance['@odata.type']:
                update_service_uri = instance['@odata.id']

    print (update_service_uri)

    update_service_response = _redfishobj.get(update_service_uri)

    path = update_service_response.obj.HttpPushUri
    print (path)

    body = []

    json_data = {'UpdateRepository': update_repo, 'UpdateTarget': update_target, 'ETag': 'atag', 'Section': 0}
    session_key = _redfishobj.session_key

    #ImageLocation = "c:\\test"
    #filename = "I41_2.40_08_10_2020.fwpkg"
    #ImagePath = os.path.join(ImageLocation, filename)
    #file_name = os.path.basename(firmware_loc)
    #print (ImagePath)
    with open(firmware_loc, 'rb') as fle:
        output = fle.read()

    session_tuple = ('sessionKey', session_key)
    parameters_tuple = ('parameters', json.dumps(json_data))
    file_tuple = ('file', (filename, output, 'application/octet-stream'))

    #Build the payload from each multipart-form data tuple
    body.append(session_tuple)
    body.append(parameters_tuple)
    body.append(file_tuple)
    #print (body)

    #Create our header dictionary
    header = {'Cookie': 'sessionKey=' + session_key}

    print ('Begin upload....please wait...')
    # We pass the whole list payload to post
    resp = _redfishobj.post(path, body, headers=header)

    if resp.status == 400:
        sys.stderr.write("Failed to upload firmware...")
    elif not resp.status in [200, 201]:
        sys.stderr.write("An http response of '%s' was returned.\n" % resp.status)
    else:
        print("Upload complete!\n")


<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 3. This is about create task function</h3>
      
1. The function create_task takes in parameters such as _redfishobj, firmware_loc, and tpm_flag.<br>
<br>
    
2. It creates a new task for applying firmware updates using the Redfish API and HTTP POST request.<br>
    <br>

3. The function first obtains the session key and creates the necessary header dictionary for the HTTP request.It then checks if there are any current tasks in the task queue before creating a new task with a random ID and the given firmware file name.<br>
    <br>

4. The task is created using the update_task dictionary and sent as an HTTP POST request to the specified path.<br>
    <br>
5.   Finally, the function returns an appropriate response based on the HTTP status code.
<br>
    



 </div>

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 4. This is Main function overview.</h3>
<b>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE TO PROCEED FURTHER<br>
    <ul>
           <li>The path to the firmware file to upload (-c/--component)</li>
        <li>The iLO IP of the server (-i/--ilo)</li>
        <li> The iLO username to login (-u/--user)</li>
        <li>The iLO password to log in (-p/--password)</li>
    </ul>
</div>

In [ ]:
def create_task(_redfishobj, firmware_loc, tpm_flag=True):

    session_key = _redfishobj.session_key
    #Create our header dictionary
    header = {'Cookie': 'sessionKey=' + session_key}

    updatable_by = ['Uefi']
    task_path = '/redfish/v1/UpdateService/UpdateTaskQueue/'
    file_name = os.path.basename(firmware_loc)

    task_resp = _redfishobj.get(task_path)
    if not task_resp.obj["Members@odata.count"]:
        print ("No current tasks, proceed to create new task")


    update_task = {'Name': 'Update-%s-%s' % (str(randint(0, 1000000)),file_name), 'Command': 'ApplyUpdate','Filename': file_name, 'UpdatableBy': updatable_by, 'TPMOverride': tpm_flag}

    resp = _redfishobj.post(task_path, update_task, headers=header)
    #print (resp)
    if resp.status == 400:
        sys.stderr.write("Failed to create update task queue...")
    elif not resp.status in [200, 201]:
        sys.stderr.write("An http response of '%s' was returned.\n" % resp.status)
    else:
        print("Task Queue Creation complete!\n")


In [2]:
if __name__ == "__main__":

    # Initialize parser
    parser = argparse.ArgumentParser(description = "Script to upload and flash NVMe FW")

    parser.add_argument(
        '-c',
        '--component',
        dest='comp_path',
        action="store",
        required=True,
        help="The path to the firmware file to upload",
        default=None)
    parser.add_argument(
        '-s',
        '--session_key',
        dest='session_key',
        action="store",
        required=False,
        help="Http session key for the server",
        default=None)
    parser.add_argument(
        '-i',
        '--ilo',
        dest='ilo_ip',
        action="store",
        required=False,
        help="iLO IP of the server",
        default=None)
    parser.add_argument(
        '-u',
        '--user',
        dest='ilo_user',
        action="store",
        required=False,
        help="iLO username to login",
        default=None)
    parser.add_argument(
        '-p',
        '--password',
        dest='ilo_pass',
        action="store",
        required=False,
        help="iLO password to log in.",
        default=None)

    options = parser.parse_args()

    system_url = "https://" + options.ilo_ip
    print (system_url)

    # Upload the firmware file to the iLO Repository
    update_repo_flag = True
    # Update the system with the firmware file
    update_target_flag = False

    comp_type = 'C'
    tpm_flag = True

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = True
    #system_url = "https://" + "15.146.46.49"
    #session_id = "515b196969f7d879886ee4a2da4ccba8"

    try:
        # Create a Redfish client object
        if options.session_key:
            redfish_obj = RedfishClient(base_url=system_url, session_key= options.session_key)
        else:
            redfish_obj = RedfishClient(base_url=system_url, username=options.ilo_user, password=options.ilo_pass)
        # Login with the Redfish client
        redfish_obj.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    upload_firmware(redfish_obj, options.comp_path, comp_type, update_repo_flag, update_target_flag)
    if comp_type == 'C':
        create_task(redfish_obj, options.comp_path, tpm_flag)

    redfish_obj.logout()

NameError: name 'ServerDownOrUnreachableError' is not defined

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library: <a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.HPE ilorest-api explorer: <a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.REST API: <a href = "https://restfulapi.net/">LINK</a><br>
    
    
</div>